# Analiza wypływu parametrów na algorytm mrówkowy

In [19]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import re
from pathlib import Path
from glob import glob
from collections import defaultdict

Przydatne funkcje

In [24]:
def load_mrowczenie(file_path: str) -> defaultdict:
    line_pattern = r'Pokolenie nr=\s*(?P<generation>\d+)\s+najlepszy=\s*(?P<best_ant_num>\d+)\(dl=\s*(?P<best_value>[-+]?[0-9]*\.?[0-9]*)\),\s*najgorszy=\s*(?P<worst_ant_num>\d+)\(dl=\s*(?P<worst_value>[-+]?[0-9]*\.?[0-9]*)\),\s*średnia_dl=\s*(?P<mean_value>[-+]?[0-9]*\.?[0-9]*)'
    
    with open(file_path, 'r', encoding='windows-1250') as file:
        lines = file.readlines()
        
#     data = []
    data = defaultdict(list)
    for line in lines:
        match = re.match(line_pattern, line)
        if not match:
            print('Line didn\'t match!')
            continue
            
        data['generacja'].append(match.group('generation'))
        data['najlepsza mrówka'].append(match.group('best_ant_num'))
        data['najlepsza wartość'].append(match.group('best_value'))
        data['najgorsza mrówka'].append(match.group('worst_ant_num'))
        data['najgorsza wartość'].append(match.group('worst_value'))
        data['średnia wartość'].append(match.group('mean_value'))
#         data.append([match.group(g) for g in ['generation', 'best_ant_num', 'best_value', 'worst_ant_num', 'worst_value', 'mean_value']])
        
    return data

In [23]:
['aa'] * 6

['aa', 'aa', 'aa', 'aa', 'aa', 'aa']

In [53]:
def load_parameter_messurements(directory: str) -> pd.DataFrame:
    directory = Path(directory)
    parameter_name = directory.stem
    parameter_dirs = sorted(d for d in directory.iterdir() if d.is_dir())
    
    data = defaultdict(list)
    for parameter_dir in parameter_dirs:
        parameter_value = parameter_dir.name
        parameter_data = load_mrowczenie(parameter_dir / Path('mrowczenie.txt'))
        for k, v in parameter_data.items():
            data[k].extend(v)
        n = len(parameter_data[list(parameter_data.keys())[0]])
        data['testowany parametr'].extend([parameter_name] * n)
        data['wartość parametru'].extend([parameter_value] * n)
        
    df = pd.DataFrame(data)
    df = df.astype({
        'generacja': int,
        'najlepsza mrówka': int,
        'najlepsza wartość': float,
        'najgorsza mrówka': int,
        'najgorsza wartość': float,
        'średnia wartość': float,
        'wartość parametru': float
    })
    return df

In [55]:
df = load_parameter_messurements('wyniki/aco_alfa')
df.info()
df['wartość parametru'].unique()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   generacja           4000 non-null   int64  
 1   najlepsza mrówka    4000 non-null   int64  
 2   najlepsza wartość   4000 non-null   float64
 3   najgorsza mrówka    4000 non-null   int64  
 4   najgorsza wartość   4000 non-null   float64
 5   średnia wartość     4000 non-null   float64
 6   testowany parametr  4000 non-null   object 
 7   wartość parametru   4000 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 250.1+ KB


,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,45,16.750883,49,21.968958,19.604503,aco_alfa,0.5
1,1,0,16.750883,8,23.613458,19.799045,aco_alfa,0.5
2,2,0,16.750883,16,21.257485,19.575276,aco_alfa,0.5
3,3,47,17.369803,34,22.419693,19.596351,aco_alfa,0.5
4,4,0,17.369803,41,21.898665,19.749420,aco_alfa,0.5
...,...,...,...,...,...,...,...,...
3995,995,0,17.654196,47,19.589807,18.676697,aco_alfa,2.0
3996,996,0,17.654196,11,19.413171,18.660359,aco_alfa,2.0
3997,997,0,17.654196,5,21.739988,18.610928,aco_alfa,2.0
3998,998,0,17.654196,41,19.589807,18.380930,aco_alfa,2.0
